In [1]:
import warnings

import numpy as np
import pandas as pd

from gensim.models.doc2vec import Doc2Vec
from sqlalchemy import create_engine
from surprise import Dataset, Reader
from tqdm import tqdm

from src.models import cf, evaluate_model
from src.utilities import utilities

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data and Models

In [2]:
# global variables
DATA_PATH = "data/evaluation"
D2V_PATH = "models/d2v"
CATEGORY = "Grocery_and_Gourmet_Food"

# d2v training parameters
EPOCHS = 10

# training parameters
N_EPOCHS = 5
LR_ALL = 0.005
BETA = 0.1

# reproducibility checks
SEED = 42
np.random.seed(42)

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")
d2v = Doc2Vec.load(f"{D2V_PATH}/{CATEGORY}_user_item_50_{EPOCHS}_d2v.model")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...
47769,77420,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A192LQZWDYPR4U,5.0,Another quality Reese Peanut Butter Cup produc...,2014-02-27,quality reese peanut butter cup product great ...
47770,77421,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A2QKXW3LDQ66P5,5.0,I purchased these for my husband who has every...,2013-02-20,purchase husband love reeses valentine day pre...
47771,77430,B00ID9VSOM,"Viva Labs Organic Coconut Sugar: Non-GMO, Low-...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2P3TGJU301KXD,5.0,this stuff is INCREDIBILY yummy! SO much bette...,2014-07-15,stuff incredibily yummy good regular brown sug...
47772,77456,B00IRL93SY,Barrie House Kenya Estate - AA Single Cup Caps...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AEFE9VDHTQ199,5.0,"Very nice aroma, body and taste! Will buy this...",2014-05-24,nice aroma body taste buy coffee good coffee a...
47773,77508,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2AEZQ3DGBBLPR,2.0,This is a no go for diabetics according to my ...,2014-06-26,diabetic accord wife doctor order intention us...


In [4]:
# testing d2v models
d2v.dv[0]

array([-2.69315392e-01,  1.77457169e-01, -1.50210857e-01,  2.73750961e-01,
       -1.25641584e-01,  1.87143400e-01, -6.93344045e-04,  6.15229197e-02,
        2.11115584e-01,  1.25968456e-01, -4.85256240e-02, -5.65706976e-02,
       -1.99691564e-01, -2.46316718e-04,  2.05525205e-01, -3.18667293e-01,
        1.79601699e-01, -5.59894741e-02, -2.77974214e-02, -3.20212483e-01,
       -4.33429256e-02, -1.83360443e-01,  7.21168593e-02,  1.21368565e-01,
       -1.03593357e-01,  3.28915179e-01, -3.99071664e-01,  5.18425256e-02,
       -3.10916811e-01, -4.83547822e-02,  2.06307508e-03,  2.12369546e-01,
       -6.04294017e-02, -3.64843979e-02,  5.01722582e-02, -9.40858275e-02,
       -2.99028754e-01, -1.37448817e-01,  4.79324944e-02,  3.00572991e-01,
       -2.44538441e-01, -4.12640929e-01, -9.99419242e-02, -1.71399504e-01,
       -1.28195733e-01, -1.09600469e-01, -2.79572457e-01,  5.46566062e-02,
        1.19243681e-01, -1.26136661e-01], dtype=float32)

# Generating User & Item Embeddings

In [5]:
user_idx_map, user_vecs, item_idx_map, item_vecs = utilities.generate_user_item_embeddings(train, d2v)

In [6]:
# converting factors into numpy obj
user_factors = user_vecs.to_numpy()
item_factors = item_vecs.to_numpy()

In [7]:
# check user factors
user_factors[0,:]

array([-0.02061913, -0.00143573, -0.0034858 ,  0.01238337, -0.00435263,
       -0.0059842 ,  0.0124501 ,  0.0046955 ,  0.01869343,  0.02591518,
        0.01818508,  0.01330373, -0.0275974 ,  0.01780992,  0.0090669 ,
       -0.00876903,  0.00522102,  0.02922117, -0.02179917, -0.02653198,
       -0.01430879, -0.01388872, -0.00422054,  0.01177809, -0.00635913,
        0.02515404, -0.02425172, -0.01412499, -0.0229231 , -0.0077565 ,
        0.01418342,  0.00071108,  0.00133703, -0.0057057 , -0.01286033,
       -0.01078283, -0.01986053,  0.01030534,  0.03729956,  0.01735871,
        0.00730524, -0.02362714,  0.00063266, -0.01385546, -0.012858  ,
       -0.00943028,  0.00276966, -0.01782037,  0.02445681, -0.01371916],
      dtype=float32)

In [8]:
# check item factors
item_factors[0,:]

array([-2.69315392e-01,  1.77457169e-01, -1.50210857e-01,  2.73750961e-01,
       -1.25641584e-01,  1.87143400e-01, -6.93344045e-04,  6.15229197e-02,
        2.11115584e-01,  1.25968456e-01, -4.85256240e-02, -5.65706976e-02,
       -1.99691564e-01, -2.46316718e-04,  2.05525205e-01, -3.18667293e-01,
        1.79601699e-01, -5.59894741e-02, -2.77974214e-02, -3.20212483e-01,
       -4.33429256e-02, -1.83360443e-01,  7.21168593e-02,  1.21368565e-01,
       -1.03593357e-01,  3.28915179e-01, -3.99071664e-01,  5.18425256e-02,
       -3.10916811e-01, -4.83547822e-02,  2.06307508e-03,  2.12369546e-01,
       -6.04294017e-02, -3.64843979e-02,  5.01722582e-02, -9.40858275e-02,
       -2.99028754e-01, -1.37448817e-01,  4.79324944e-02,  3.00572991e-01,
       -2.44538441e-01, -4.12640929e-01, -9.99419242e-02, -1.71399504e-01,
       -1.28195733e-01, -1.09600469e-01, -2.79572457e-01,  5.46566062e-02,
        1.19243681e-01, -1.26136661e-01], dtype=float32)

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [9]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,2,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1TCSC0YWT82Q0,5.0,I love ethnic foods and to cook them. I recent...,2013-08-03,love ethnic food cook recently purchase produc...
1,8,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1Z7Y2GMAP9SRY,5.0,I like to make my own curry but this is a tast...,2014-06-27,like curry tasty alternative use base kind dif...
2,23,B00004S1C5,"Ateco Food Coloring Kit, 6 colors","['Grocery & Gourmet Food', 'Cooking & Baking',...",A14YSMLYLJEMET,1.0,This product is no where near natural / organi...,2013-03-29,product near natural organic wish review purch...
3,31,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",A2F488C4PLWGEI,5.0,If my wife drinks a cup of this tea when she f...,2014-03-23,wife drink cup tea feel attack come help avoid...
4,32,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AO1HXV7DWZZIR,5.0,I don't know about the medicinal aspects of th...,2014-02-06,know medicinal aspect tea flavor downright scr...
28001,77519,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A1WT3TVHANP7ZF,3.0,Hmmm. I really wanted to love this sweetener. ...,2014-07-22,hmmm want love sweetener half sugar half stevi...
28002,77520,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A3NEAETOSXDBOM,5.0,"I confess I have a sweet tooth, and love the t...",2014-06-30,confess sweet tooth love taste sugar recognize...
28003,77521,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",AD1ZOPB0BBEHB,4.0,"It has a little of the stevia aftertaste, but ...",2014-07-17,little stevia aftertaste fair compromise able ...
28004,77522,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A18ECVX2RJ7HUE,5.0,i love marinade for grilled flank steak or lon...,2014-05-30,love marinade grilled flank steak london broil...
28005,77523,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2G04D4QZAXL15,3.0,I've been using Truvia (a form of stevia) on m...,2014-05-27,use truvia form stevia cereal greek yogurt yea...


In [10]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [11]:
print(test_user_history)

                  reviewerID  \
0      A00177463W0XWB16A9O05   
1      A022899328A0QROR32DCT   
2      A068255029AHTHDXZURNU   
3      A06944662TFWOKKV4GJKX   
4             A1004703RC79J9   
...                      ...   
13274          AZWRZZAMX90VT   
13275          AZXKAH2DE6C8A   
13276          AZXON596A1VXC   
13277          AZYXC63SS008M   
13278          AZZ5ASC403N74   

                                                    asin  
0                               [B00474OR8G, B00BFM6OAW]  
1                                           [B00CMQDKES]  
2                                           [B001FA1K2G]  
3                                           [B000GFYRHG]  
4                                           [B003GTR8IO]  
...                                                  ...  
13274  [B0007R9L4M, B000CN7BMA, B001EQ5D1K, B002VT3GX...  
13275   [B000MAK41I, B004X8TJP2, B006H34CUS, B007W14RMM]  
13276                           [B001EO5S0I, B00271QQ7Q]  
13277                    

## Preparing Dataset for Surprise's Algorithm

In [12]:
# create reader
reader = Reader(rating_scale=(1,5))
# generate data required for surprise
data = Dataset.load_from_df(train[["reviewerID", "asin", "overall"]], reader)
# generating trainset
trainset = data.build_full_trainset()

## Instantiate Pre-Initialised Matrix Factorization (Paragraph Vector)

In [13]:
# instantiating mod-ecf
mod_ecf = cf.PreInitialisedMF(user_map=user_idx_map,
                              item_map=item_idx_map,
                              user_factor=user_factors,
                              item_factor=item_factors,
                              learning_rate=LR_ALL,
                              beta=BETA,
                              num_epochs=N_EPOCHS,
                              num_factors=50)

In [14]:
%%time
# fitting to training data
mod_ecf.fit(trainset, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
CPU times: user 2min 15s, sys: 2.62 s, total: 2min 18s
Wall time: 2min 21s


In [15]:
%%time
# generate candidate items for user to predict rating
testset = trainset.build_anti_testset()

CPU times: user 34 s, sys: 2.12 s, total: 36.1 s
Wall time: 37 s


In [16]:
%%time
# predict ratings for all pairs (u, i) that are NOT in the training set
candidate_items = mod_ecf.test(testset, verbose=False)

CPU times: user 8min 48s, sys: 1min 59s, total: 10min 48s
Wall time: 11min 34s


## Loop through N = {10, 25, 30, 45}

In [17]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [18]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    # top_ns = get_top_n(candidate_items, n)
    top_ns = mod_ecf.get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'MOD-ECF',
                                                    top_ns = top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n,
                                                    mf_based = True))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The MOD-ECF has an average recall@10: 0.00491, average novelty@10: 0.96825
The MOD-ECF has an average recall@25: 0.01203, average novelty@25: 0.96812
The MOD-ECF has an average recall@30: 0.01521, average novelty@30: 0.96805
The MOD-ECF has an average recall@45: 0.02190, average novelty@45: 0.96809


# Evaluate N-Recommendations

## N=10

In [19]:
top_ns_10 = n_recommendations[10][0]
utilities.retrieve_recommendations(train, top_ns_10, mf_based=True)

For user: A2H84V55USFJQX:
Purchase History:
             asin                                              title
13844  B000ZSZ5S4  Blue Diamond Almonds, Bold Salt &amp; Vinegar,...
41088  B005K4Q1T0  Grove Square Hot Cocoa Dark Chocolate, 24 Sing...

Recommending:

         asin                                              title
0  B004YTV5S4  Navitas Organics Hemp Seeds, 8 oz. Bags (Pack ...
1  B004IX03MK  Wedderspoon 100% Raw Organic Dandelion Honey, ...
2  B00856TSCC  Manitoba Harvest Hemp Hearts Raw Shelled Hemp ...
3  B00B8DSFXC  Kevala Organic Black Raw and Unhulled Sesame S...
4  B004334FPG        Bob's Red Mill White Sesame Seeds, 16-ounce
5  B003BI2EUW     YS BEE FARMS Raw Tupelo Blossom Honey, 13.5 OZ
6  B007JBO41E  Emergency Preparedness &amp; Survival Gluten-F...
7  B001EQ5G1C  Manitoba Harvest Organic Hemp Hearts Raw Shell...
8  B00B8DU1QQ           Kevala Organic Toasted Sesame Seeds 2Lbs
9  B004TDTZEG  MW Polar Herring, Hot Tomato Sauce, 6-Ounce (P...


## N=25

In [20]:
top_ns_25 = n_recommendations[25][0]
utilities.retrieve_recommendations(train, top_ns_25, mf_based=True)

For user: A2W9B725TZBXOX:
Purchase History:
             asin                                              title
1645   B0005Z6LLW  Ghirardelli Hot Chocolate Mix , Chocolate Moch...
4683   B000ED9L6C  Bob's Red Mill Raw Shelled Sunflower Seeds (Ke...
8508   B000GATCRQ  Dream Foods International Volcano Lemon Burst,...
15333  B0014EOUAW  V8 V-Fusion Light Peach Mango Juice Drink, 46-...
18192  B001E52ZAS  Post Shredded Wheat Lightly Frosted Cereal, Sp...
21093  B001EPQV1W  Honey Bunches of Oats with Almonds, 14.5-Ounce...
22932  B001G8UC8K  Tootsie Roll Midgees Candy 5 Pound Value Bag 7...
23025  B001GVIRD4                     Grocery &amp; Gourmet Food" />
24945  B001NC8HQS  Idahoan Buttery homestyle flavored mashed pota...
25830  B001SAQ7WQ  Progresso Traditional Soup, Chickarina (Chicke...
26956  B0025UOMY8  Maruchan Yakisoba Spicy Vegetable, 3.79-Ounce ...
33467  B00421DMLM  Libby's Splenda Sliced Peaches, 23.5-Ounce Jar...
34141  B004727CL2  Snack Factory Pretzel Crisps Everything,

## N=30

In [21]:
top_ns_30 = n_recommendations[30][0]
utilities.retrieve_recommendations(train, top_ns_30, mf_based=True)

For user: A1G8ECTOF35O2R:
Purchase History:
             asin                                              title
9115   B000H26J7E  Lindt Excellence Bar, 70% Cocoa Smooth Dark Ch...
47677  B00EKLPLU4              Healthworks Cacao Powder Organic, 1lb

Recommending:

          asin                                              title
0   B00028Q45A    Arrowhead Mills Organic Flax Seeds, 1 Pound Bag
1   B000HLH7AS          Mr Kipling Cakes Mini Battenbergs- 5 pack
2   B000IXUJOQ  ALMOND JOY, Chocolate Coconut Candy Bar (Pack ...
3   B000MIFS3Y  Chef Boyardee Beef Ravioli, 14.25-Ounce Microw...
4   B001EQ5G1C  Manitoba Harvest Organic Hemp Hearts Raw Shell...
5   B001SJHNKC  Kicking Horse Organic Whole Bean Coffee, Cliff...
6   B0029JASWA  Dove Dark Chocolate Promises, 9.5-Ounce Packag...
7   B0029J6QLM         Snickers Dark Chocolate Candy (Pack of 24)
8   B002NHSQ6I  Newman's Own Organics Organic Premium Chocolat...
9   B002RBRY0Y  Lindt LINDOR Dark Chocolate Truffles, Kosher, ...
10  B00

## N=45

In [22]:
top_ns_45 = n_recommendations[45][0]
utilities.retrieve_recommendations(train, top_ns_45, mf_based=True)

For user: A1NEKFNO1E743Y:
Purchase History:
            asin                                              title
8695  B000GFYRHG  Bigelow Constant Comment Tea 20-Count Boxes (P...

Recommending:

          asin                                              title
0   B004YTV5S4  Navitas Organics Hemp Seeds, 8 oz. Bags (Pack ...
1   B004IX03MK  Wedderspoon 100% Raw Organic Dandelion Honey, ...
2   B00856TSCC  Manitoba Harvest Hemp Hearts Raw Shelled Hemp ...
3   B00B8DSFXC  Kevala Organic Black Raw and Unhulled Sesame S...
4   B004334FPG        Bob's Red Mill White Sesame Seeds, 16-ounce
5   B007JBO41E  Emergency Preparedness &amp; Survival Gluten-F...
6   B003BI2EUW     YS BEE FARMS Raw Tupelo Blossom Honey, 13.5 OZ
7   B004TDTZEG  MW Polar Herring, Hot Tomato Sauce, 6-Ounce (P...
8   B00B8DU1QQ           Kevala Organic Toasted Sesame Seeds 2Lbs
9   B008XCSXHO  Vitacost Organic Apple Cider Vinegar with ''Mo...
10  B001EQ5G1C  Manitoba Harvest Organic Hemp Hearts Raw Shell...
11  B00DDT11

# Cross-Analysis for Cold-Start Users (<= 2 Purchased Items)

In [23]:
cold_start_users = utilities.generate_cold_start_users(train)

In [24]:
for n in tuple(zip([10, 25, 30, 45], [top_ns_10, top_ns_25, top_ns_30, top_ns_45])):
    cold_start_top_ns = dict(filter(lambda x: x[0] in cold_start_users, n[1].items()))
    # evaluate how well the recommended items predicted the future purchases
    # on cold start users
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'MOD-ECF',
                                                    top_ns = cold_start_top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n[0],
                                                    mf_based = True))

The MOD-ECF has an average recall@10: 0.00488, average novelty@10: 0.97484
The MOD-ECF has an average recall@25: 0.01244, average novelty@25: 0.97276
The MOD-ECF has an average recall@30: 0.01672, average novelty@30: 0.97238
The MOD-ECF has an average recall@45: 0.02381, average novelty@45: 0.97192


# Generating Recommended Items DataFrame

In [25]:
max_recommendations = (utilities
                       .generate_recommendations_df(
                           train, 
                           n_recommendations, 
                           "MOD-ECF", mf_based=True))

In [26]:
 max_recommendations

,reviewerID,item_rank,asin,algorithm,title
0,A00177463W0XWB16A9O05,0,B00028Q45A,MOD-ECF,"Arrowhead Mills Organic Flax Seeds, 1 Pound Bag"
1,A00177463W0XWB16A9O05,1,B001EQ5G1C,MOD-ECF,Manitoba Harvest Organic Hemp Hearts Raw Shell...
2,A00177463W0XWB16A9O05,2,B003BI2EUW,MOD-ECF,"YS BEE FARMS Raw Tupelo Blossom Honey, 13.5 OZ"
3,A00177463W0XWB16A9O05,3,B004334FPG,MOD-ECF,"Bob's Red Mill White Sesame Seeds, 16-ounce"
4,A00177463W0XWB16A9O05,4,B004IX03MK,MOD-ECF,"Wedderspoon 100% Raw Organic Dandelion Honey, ..."
...,...,...,...,...,...
602860,AZZ5ASC403N74,40,B003S1TNIS,MOD-ECF,"FiberGourmet Light Spaghetti, 8-Ounce Boxes (P..."
602861,AZZ5ASC403N74,41,B005CULDQI,MOD-ECF,"York Peppermint Patties, 175-Count Changemaker..."
602862,AZZ5ASC403N74,42,B005QKH1V0,MOD-ECF,"Cameron's Coffee Roasted Ground Coffee, Variet..."
602863,AZZ5ASC403N74,43,B001EQ5D6U,MOD-ECF,"Fiber One Caramel Delight Cereal, 15.6-Ounce B..."


# Store in `SQLite` DB

In [29]:
engine = create_engine("sqlite:///recommender.db", echo=True)

In [30]:
max_recommendations.to_sql(f"{CATEGORY}", con=engine, if_exists="append")

2021-09-29 20:07:47,798 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Grocery_and_Gourmet_Food")
2021-09-29 20:07:47,803 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-29 20:07:48,074 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 20:07:52,651 INFO sqlalchemy.engine.Engine INSERT INTO "Grocery_and_Gourmet_Food" ("index", "reviewerID", item_rank, asin, algorithm, title) VALUES (?, ?, ?, ?, ?, ?)
2021-09-29 20:07:52,652 INFO sqlalchemy.engine.Engine [generated in 3.54790s] ((0, 'A00177463W0XWB16A9O05', 0, 'B00028Q45A', 'MOD-ECF', 'Arrowhead Mills Organic Flax Seeds, 1 Pound Bag'), (1, 'A00177463W0XWB16A9O05', 1, 'B001EQ5G1C', 'MOD-ECF', 'Manitoba Harvest Organic Hemp Hearts Raw Shelled Hemp Seeds, 12 Ounce (Pack of 2)'), (2, 'A00177463W0XWB16A9O05', 2, 'B003BI2EUW', 'MOD-ECF', 'YS BEE FARMS Raw Tupelo Blossom Honey, 13.5 OZ'), (3, 'A00177463W0XWB16A9O05', 3, 'B004334FPG', 'MOD-ECF', "Bob's Red Mill White Sesame Seeds, 16-ounce"), (4, 'A00177463W0XWB16A9O05', 4, 'B0